In [6]:
import pandas as pd
import sklearn.metrics as metr
import numpy as np
from ignite.metrics import Precision
import torch
from torchvision import models

In [98]:
y_pred = np.array([[1,0,1],
                   [1,1,1],
                   [0,1,0]])
y = np.array([[1,1,1],
              [0,0,0],
              [1,0,1]])
metr.precision_recall_fscore_support(y, y_pred, average='macro')

(0.3333333333333333, 0.3333333333333333, 0.3333333333333333, None)

In [99]:
prec = Precision(average=True, is_multilabel=True)
torch_y, torch_y_pred = torch.from_numpy(y), torch.from_numpy(y_pred)
prec.update((torch_y_pred, torch_y))
prec.compute()

0.3333333333333333

In [13]:
m = models.resnet50(pretrained=True)
for param in m.parameters():
    param.requires_grad = False
input_num = 3
out_shape = 2
m.conv1 = torch.nn.Conv2d(input_num, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
last_layer_in_channels = m.fc.in_features
m.fc = torch.nn.Linear(last_layer_in_channels, out_shape)

In [70]:
device = 'cuda'
model_path = '../Saved_models/'
model_name = 'Saved_models'
model = m
model.load_state_dict(torch.load(model_path+model_name))
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [87]:
from ignite.metrics import Precision, Recall, Accuracy

def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y


class Metrics:
    def __init__(self):

        self.acc = Accuracy(is_multilabel=True)
        self.prec = Precision(average=True, is_multilabel=True)
        self.rec = Recall(average=True, is_multilabel=True)

    def update(self, outputs, labels_batch):

        outputs, labels_batch = thresholded_output_transform((outputs, labels_batch))
        print(outputs, labels_batch)
        self.acc.update((outputs, labels_batch))
        self.prec.update((outputs, labels_batch))
        self.rec.update((outputs, labels_batch))

    def reset(self):

        self.acc.reset()
        self.prec.reset()
        self.rec.reset()

    def compute(self):

        return {'accuracy': self.acc.compute(),
                'precision': self.prec.compute(),
                'recall': self.rec.compute()
                }

In [71]:
import os
import torchvision.transforms.functional as TF
from keras.preprocessing.image import array_to_img, img_to_array
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
import h5py


def load_image(file_name, type='image'):
    f = h5py.File(file_name, 'r')
    file_np = f['img'][()]
    if type == 'image':
        file_np = (file_np / 255.0).astype('float32')
    elif type == 'mask':
        file_np = file_np.astype('uint8')
    else:
        print('not choosed type to load')
        return
    return file_np


class MyDataset(Dataset):

    def __init__(self, train_test_id, labels_ids, args, train):

        self.train_test_id = train_test_id
        self.labels_ids = labels_ids
        self.image_path = args['image_path']
        self.attribute = args['attribute']
        self.mask_use = args['mask_use']
        self.train = train
        self.all_attributes = ['attribute_globules', 'attribute_milia_like_cyst', 'attribute_negative_network',
                               'attribute_pigment_network', 'attribute_streaks']

        self.indexes = [i for i, val in enumerate(self.all_attributes) for attr in self.attribute if attr == val]

        if train == 'valid':
            self.labels_ids = self.labels_ids[self.train_test_id['Split'] != 'train'].values.astype('uint8')
            self.train_test_id = self.train_test_id[self.train_test_id['Split'] != 'train'].ID.values
            #print('Train =', self.train, 'train_test_id.shape: ', self.train_test_id.shape)

        self.n = self.train_test_id.shape[0]

    def __len__(self):
        return self.n


    def __getitem__(self, index):

        name = self.train_test_id[index]
        path = self.image_path
        # Load image and from h5
        image = load_image(os.path.join(path, '%s.h5' % name), 'image')
        mask = np.empty([image.shape[0], image.shape[1], len(self.attribute)])
        if self.mask_use:
            for i, attr in enumerate(self.attribute):
                mask[:, :, i] = load_image(os.path.join(path, '{}_{}.h5'.format(name, attr)), 'mask')[:, :, 0]
        else:
            mask = image

        if self.mask_use:
            if self.train == 'valid':
                mask.fill(0.)
            image_with_mask = np.dstack((image, mask))
        else:
            image_with_mask = image

        labels = np.array([self.labels_ids[index, ind] for ind in self.indexes])

        return image_with_mask, labels, name


def make_loader(train_test_id, labels_ids, args, train=True, shuffle=True):

    data_set = MyDataset(train_test_id=train_test_id,
                         labels_ids=labels_ids,
                         args=args,
                         train=train)
    data_loader = DataLoader(data_set,
                             batch_size=args['batch_size'],
                             shuffle=shuffle
                             )
    return data_loader

In [114]:
y_pred = np.around([[-1.4553,  1.4663],
        [-1.1743,  1.1686],
        [-1.5759,  1.7062],
        [-1.1638,  0.6727]])
print(y_pred)
y = np.array([[0., 1.],
        [0., 1.],
        [1., 1.],
        [1., 0.]])
print(metr.accuracy_score(y, y_pred))
metr.precision_recall_fscore_support(y, y_pred, average='macro')

[[-1.  1.]
 [-1.  1.]
 [-2.  2.]
 [-1.  1.]]


ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass-multioutput targets

In [120]:
y_pred = np.around([[0,  1],
        [0,  1]])
y = np.array([[0., 1.],
        [0., 0.]])
print(metr.accuracy_score(y.ravel(), y_pred.ravel()))
metr.precision_recall_fscore_support(y, y_pred, average='macro')

0.75


(0.25, 0.5, 0.3333333333333333, None)

In [113]:
prec = Precision(average=True, is_multilabel=True)
rec = Recall(average=True, is_multilabel=True)
acc = Accuracy(is_multilabel=True)
torch_y, torch_y_pred = torch.from_numpy(y), torch.from_numpy(y_pred)
print(torch_y_pred)
prec.update((torch_y_pred, torch_y))
rec.update((torch_y_pred, torch_y))
acc.update((torch_y_pred, torch_y))
print(rec.compute())
print(acc.compute())
print(prec.compute())
rec.reset()
prec.reset()
acc.reset()

tensor([[0, 1],
        [0, 1]])
0.5
0.5
0.5


In [93]:
train_test_id = pd.read_csv('../Data/train_test_id.csv')
mask_ind = pd.read_csv('../Data/mask_ind.csv')
metric = Metrics()
args = {
    'image_path': '../Data/h5/',
    'attribute':['attribute_pigment_network', 'attribute_milia_like_cyst'],
    'mask_use':False,
    'batch_size': 2
}
loader = make_loader(train_test_id, mask_ind, args, train='valid', shuffle=True)
for i, (image_batch, labels_batch, names) in enumerate(loader):
    image_batch = image_batch.permute(0, 3, 1, 2).to(device).type(torch.cuda.FloatTensor)
    labels_batch = labels_batch.to(device).type(torch.cuda.FloatTensor)

    output_probs = model(image_batch)
    #print(output_probs, labels_batch)
    outputs = torch.sigmoid(output_probs)
    metric.update(outputs, labels_batch)
    metrics = metric.compute()
    print('Acc: {:.4f} Prec: {:.4f} Recall: {:.4f}'.format(
            metrics['accuracy'], metrics['precision'], metrics['recall']))
metric.reset()

tensor([[0., 1.],
        [0., 1.]], device='cuda:0', grad_fn=<RoundBackward>) tensor([[0., 1.],
        [0., 0.]], device='cuda:0')
Acc: 0.5000 Prec: 0.5000 Recall: 0.5000
tensor([[0., 1.],
        [0., 1.]], device='cuda:0', grad_fn=<RoundBackward>) tensor([[0., 1.],
        [1., 0.]], device='cuda:0')
Acc: 0.5000 Prec: 0.5000 Recall: 0.5000
tensor([[0., 1.],
        [0., 1.]], device='cuda:0', grad_fn=<RoundBackward>) tensor([[1., 0.],
        [0., 1.]], device='cuda:0')
Acc: 0.5000 Prec: 0.5000 Recall: 0.5000
tensor([[0., 1.],
        [0., 1.]], device='cuda:0', grad_fn=<RoundBackward>) tensor([[0., 1.],
        [0., 1.]], device='cuda:0')
Acc: 0.6250 Prec: 0.6250 Recall: 0.6250
tensor([[0., 1.],
        [0., 1.]], device='cuda:0', grad_fn=<RoundBackward>) tensor([[0., 0.],
        [0., 1.]], device='cuda:0')
Acc: 0.6000 Prec: 0.6000 Recall: 0.6000
tensor([[0., 1.],
        [0., 1.]], device='cuda:0', grad_fn=<RoundBackward>) tensor([[1., 1.],
        [0., 1.]], device='cuda:0')
Ac

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 2.95 GiB total capacity; 1.93 GiB already allocated; 6.88 MiB free; 21.46 MiB cached)